In [22]:
import numpy as np

# load the saved array
# my_array = np.load('pred_RAS.npy')
my_array = np.load('pred_test.npy')


In [23]:
import torch
import torch.nn as nn

import random
import math
import numpy as np
import matplotlib.pyplot as plt
import music21
from music21 import * 

In [24]:
#Vocab


NOTE_SIZE = 128
DUR_SIZE = 160
TIM_SIZE = 1000
VEL_SIZE = 128


NOTE_TOKS = [f'n{i}' for i in range(NOTE_SIZE)] 
DUR_TOKS = [f'd{i}' for i in range(DUR_SIZE)]
TIM_TOKS = [f't{i}' for i in range(TIM_SIZE)]
VEL_TOKS = [f'v{i}' for i in range(VEL_SIZE)]

BOS_TOK = "BOS"
# Le token dummy sert seulement à initialiser les mots du vocab à partir de l'index 1, conformément aux prérequis de la fonction vocab()
VOCAB = ["dummy"] + [BOS_TOK] + NOTE_TOKS + DUR_TOKS + TIM_TOKS + VEL_TOKS 

DICT = [(element, index) for index, element in enumerate(VOCAB)]


In [25]:
from torchtext.vocab import vocab
from collections import OrderedDict

custom_vocab = vocab(OrderedDict(DICT))
itos_vocab = custom_vocab.get_itos()

In [26]:
array_tokens = [itos_vocab[el] for el in my_array]
array_tokens = array_tokens[3:]
array_tokens

['n65',
 'd2',
 't2',
 'v113',
 'n62',
 'd2',
 't2',
 'v113',
 'n60',
 'd2',
 't1',
 'v108',
 'n58',
 'd2',
 't2',
 'v113',
 'n58',
 'd2',
 't2',
 'v113',
 'n58',
 'd2',
 't2',
 'v113',
 'n58',
 'd2',
 't2',
 't4',
 'n61',
 'd2',
 't2',
 'v113',
 'n58',
 'd1',
 't2',
 'v102',
 'n58',
 'd2',
 't2',
 'v113',
 'n58',
 'd1',
 't2',
 'v101',
 'n58',
 'd2',
 't2',
 'v101',
 'n58',
 'd2',
 't2',
 'v113',
 'n58',
 'd1',
 't2',
 'v112',
 'n58',
 'd1',
 't2',
 'v102',
 'n60',
 'd1',
 't2',
 'v102',
 'n58',
 'd1',
 't2',
 'v112',
 'n58',
 'd2',
 't2',
 'v102',
 'n60',
 'd1',
 't2',
 'v102',
 'n58',
 'd1',
 't2',
 'v102',
 'n62',
 'd2',
 't2',
 'v102',
 'n58',
 'd1',
 't2',
 'v114',
 'n58',
 'd2',
 't2',
 'v102',
 'n60',
 'd1',
 't2',
 'v102',
 'n58',
 'd2',
 't2',
 'v114',
 'n58',
 'd2',
 't2',
 'v114',
 'n60',
 'd2',
 't2',
 'v101',
 'n58',
 'd2',
 't2',
 'v114',
 'n58',
 'd1',
 't2',
 'v102',
 'n58']

In [27]:
len(array_tokens)//3

39

In [28]:

# Define a function to convert an array of tokens to a music21 Stream
def tokens_to_stream(token_array):
    # Initialize variables to keep track of note properties
    current_pitch = None
    current_duration = None
    current_offset = 0.0
    current_velocity = None
    
    # Create an empty stream
    stream = music21.stream.Stream()

    # Iterate through the token array and create notes
    for token in token_array:
        if token.startswith("n"):
            # Token represents a pitch
            current_pitch = int(token[1:])
        elif token.startswith("d"):
            # Token represents a duration
            current_duration = int(token[1:])
        elif token.startswith("t"):
            # Token represents a time shift
            current_offset += int(token[1:])/4
        elif token.startswith("v"):
            # Token represents a velocity
            current_velocity = int(token[1:])
        else:
            # Token is not recognized, skip
            continue

        # If all note properties have been set, create a note and add it to the stream
        if current_pitch is not None and current_duration is not None and current_velocity is not None:
            # Create a note object with the current properties
            note = music21.note.Note()
            pitch = music21.pitch.Pitch()
            pitch.midi = current_pitch
            note.pitch = pitch
            note.duration = music21.duration.Duration(current_duration/4)
            note.volume.velocity = current_velocity
            note.offset = current_offset

            # Add the note to the stream
            stream.append(note)

            # Reset note properties
            current_pitch = None
            current_duration = None
            current_velocity = None

    return stream

# Define a function to convert an array of tokens to a MIDI file
def tokens_to_midi(token_array, filename):
    # Convert the token array to a music21 Stream
    stream = tokens_to_stream(token_array)

    # Create a MIDI file from the Stream and save it
    midi_file = music21.midi.translate.streamToMidiFile(stream)
    midi_file.open(filename, "wb")
    midi_file.write()
    midi_file.close()


In [29]:
tokens_to_midi(array_tokens, 'out3.mid')